In [58]:
import numpy as np
from collections import defaultdict
from typing import List
import random

In [59]:
class VRPEnvironment:
    def __init__(self, num_vehicles, capacity, customers, distance_matrix):
        self.num_vehicles = num_vehicles
        self.capacity = capacity
        self.customers = customers
        self.distance_matrix = distance_matrix
        self.reset()
        
    def reset(self):
        self.vehicle_positions = [0] * self.num_vehicles
        self.customer_states = [tuple(self.customers)] * self.num_vehicles
        self.demands = [0] * self.num_vehicles
        self.total_distance = 0
        self.steps = 0
        
    def step(self, actions):
        rewards = [0] * self.num_vehicles
        next_customer_states = [None] * self.num_vehicles
        for i, action in enumerate(actions):
            if action < len(self.customer_states[i]):
                customer = self.customer_states[i][action]
                if self.demands[i] + customer[1] <= self.capacity:
                    self.demands[i] += customer[1]
                    rewards[i] -= self.distance_matrix[self.vehicle_positions[i]][customer[0]]
                    self.total_distance += self.distance_matrix[self.vehicle_positions[i]][customer[0]]
                    self.vehicle_positions[i] = customer[0]
                    next_customer_states[i] = self.customer_states[i][:action] + self.customer_states[i][action+1:]
        done = (len(next_customer_states[0]) == 0)
        state = tuple(self.vehicle_positions + [self.demands[0]] + [tuple(x) for x in next_customer_states])
        return state, sum(rewards), done
    



In [60]:
def train_vrp_agent(num_vehicles, capacity, customers, distance_matrix, num_episodes=1000, alpha=0.1, gamma=0.9, epsilon=0.1):
    num_customers = len(customers)

    # Initialize the Q-table with a shape that can hold all possible states and actions
    max_num_states = num_customers ** num_vehicles
    max_num_actions = capacity + 1
    q_table = np.zeros((max_num_states, max_num_actions))

    # Train the agent for the specified number of episodes
    for episode in range(num_episodes):
        # Reset the environment for each episode
        state = (0,)*num_vehicles
        total_reward = 0

        # Loop over all customers and choose an action for each vehicle
        for customer_index in range(num_customers):
            for vehicle_index in range(num_vehicles):
                # Convert the state to an index in the Q-table
                state_index = sum([state[i] * (num_customers ** i) for i in range(num_vehicles)])

                # Choose an action using epsilon-greedy exploration
                if random.uniform(0, 1) < epsilon:
                    action = random.randint(0, capacity)
                else:
                    action = np.argmax(q_table[state_index])

                # Calculate the reward for the chosen action
                customer = customers[customer_index]
                if action >= customer[1]:
                    reward = customer[1] - distance_matrix[state[vehicle_index]][customer_index]
                else:
                    reward = -100

                # Convert the next state to an index in the Q-table
                next_state = state[:vehicle_index] + (customer_index,) + state[vehicle_index+1:]
                next_state_index = sum([next_state[i] * (num_customers ** i) for i in range(num_vehicles)])

                # Update the Q-table using Q-learning
                q_table[state_index][action] += alpha * (reward + gamma * np.max(q_table[next_state_index]) - q_table[state_index][action])
                total_reward += reward

                # Move to the next state
                state = next_state

        # Print the total reward for the episode
        print("Episode:", episode, "Total reward:", total_reward)

    # Return the Q-table
    return q_table
    

In [90]:
def test_vrp_agent(q_table, num_vehicles, capacity, customers, distance_matrix, num_episodes=1000, alpha=0.1, gamma=0.9, epsilon=0.1):
    total_distance = 0
    # Loop through all vehicles
    for vehicle_idx in range(num_vehicles):
        load = 0
        distance = 0
        # Set the starting state for the current vehicle
        state = 0  # Starting depot node
        # Create a list of unvisited nodes (excluding the starting depot node)
        unvisited_nodes = list(range(1, len(customers)))
        # Loop until the route for the current vehicle is complete
        while True:
            # Choose an action for the current state using the Q-table
            q_values = q_table[state]
            valid_actions = np.nonzero(q_values)[0]
            action = random.choice(valid_actions)
            # Get the next state and reward based on the chosen action
            next_state = action
            reward = 0
            if next_state in unvisited_nodes:
                # Calculate the load and distance for the state transition
                load += customers[next_state]
                if load > capacity:
                    reward = -1  # Exceeding capacity is penalized
                    load -= customers[next_state]
                    next_state = state  # Stay at the current state
                else:
                    reward = 1  # Visiting a customer is rewarded
                    distance += distance_matrix[state][next_state]
                    unvisited_nodes.remove(next_state)
            else:
                # Penalize visiting an already visited node
                reward = -1
                next_state = state  # Stay at the current state
            # Update the Q-table for the current state and action
            old_value = q_table[state, action]
            next_max = np.max(q_table[next_state])
            new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
            q_table[state, action] = new_value
            # Move to the next state
            state = next_state
            # Check if the route is complete
            if not unvisited_nodes:
                # Return to the starting depot node
                distance += distance_matrix[state][0]
                break
            # Check if the current state and action are invalid
            if state >= len(distance_matrix) or next_state >= len(distance_matrix[state]):
                distance = float('inf')
                break
        # Add the distance traveled by the current vehicle to the total distance
        total_distance += distance
    return total_distance


In [91]:
num_vehicles = 2
capacity = 100
customers = [(1, 20), (2, 30), (3, 50), (4, 10), (5, 40)]
distance_matrix = [
    [0, 10, 20, 5, 15],
    [10, 0, 15, 20, 25],
    [20, 15, 0, 10, 30],
    [5, 20, 10, 0, 10],
    [15, 25, 30, 10, 0],
]


In [92]:
q_table = train_vrp_agent(num_vehicles, capacity, customers, distance_matrix, num_episodes=1000)


Episode: 0 Total reward: -1000
Episode: 1 Total reward: -635
Episode: 2 Total reward: -865
Episode: 3 Total reward: -735
Episode: 4 Total reward: -500
Episode: 5 Total reward: -600
Episode: 6 Total reward: -730
Episode: 7 Total reward: -260
Episode: 8 Total reward: -360
Episode: 9 Total reward: -260
Episode: 10 Total reward: -360
Episode: 11 Total reward: -140
Episode: 12 Total reward: -140
Episode: 13 Total reward: -40
Episode: 14 Total reward: 80
Episode: 15 Total reward: -55
Episode: 16 Total reward: 80
Episode: 17 Total reward: 210
Episode: 18 Total reward: 210
Episode: 19 Total reward: 210
Episode: 20 Total reward: 210
Episode: 21 Total reward: 210
Episode: 22 Total reward: 80
Episode: 23 Total reward: 210
Episode: 24 Total reward: 210
Episode: 25 Total reward: 210
Episode: 26 Total reward: 210
Episode: 27 Total reward: 210
Episode: 28 Total reward: 210
Episode: 29 Total reward: 75
Episode: 30 Total reward: 210
Episode: 31 Total reward: 80
Episode: 32 Total reward: 210
Episode: 33

Episode: 469 Total reward: 210
Episode: 470 Total reward: 210
Episode: 471 Total reward: 210
Episode: 472 Total reward: 90
Episode: 473 Total reward: 210
Episode: 474 Total reward: 210
Episode: 475 Total reward: 210
Episode: 476 Total reward: 210
Episode: 477 Total reward: 210
Episode: 478 Total reward: 210
Episode: 479 Total reward: 210
Episode: 480 Total reward: 210
Episode: 481 Total reward: 210
Episode: 482 Total reward: 210
Episode: 483 Total reward: 210
Episode: 484 Total reward: 210
Episode: 485 Total reward: 80
Episode: 486 Total reward: 210
Episode: 487 Total reward: 80
Episode: 488 Total reward: -30
Episode: 489 Total reward: 210
Episode: 490 Total reward: 210
Episode: 491 Total reward: 210
Episode: 492 Total reward: 80
Episode: 493 Total reward: 210
Episode: 494 Total reward: 210
Episode: 495 Total reward: 90
Episode: 496 Total reward: 90
Episode: 497 Total reward: 90
Episode: 498 Total reward: 90
Episode: 499 Total reward: 210
Episode: 500 Total reward: 75
Episode: 501 Tota

Episode: 969 Total reward: 210
Episode: 970 Total reward: 90
Episode: 971 Total reward: 75
Episode: 972 Total reward: 75
Episode: 973 Total reward: 210
Episode: 974 Total reward: 90
Episode: 975 Total reward: 210
Episode: 976 Total reward: 210
Episode: 977 Total reward: 80
Episode: 978 Total reward: 210
Episode: 979 Total reward: 210
Episode: 980 Total reward: 210
Episode: 981 Total reward: 210
Episode: 982 Total reward: 90
Episode: 983 Total reward: 90
Episode: 984 Total reward: 75
Episode: 985 Total reward: 210
Episode: 986 Total reward: 210
Episode: 987 Total reward: 210
Episode: 988 Total reward: 210
Episode: 989 Total reward: 75
Episode: 990 Total reward: 210
Episode: 991 Total reward: 210
Episode: 992 Total reward: 210
Episode: 993 Total reward: 210
Episode: 994 Total reward: 210
Episode: 995 Total reward: 210
Episode: 996 Total reward: 90
Episode: 997 Total reward: 210
Episode: 998 Total reward: 210
Episode: 999 Total reward: 75


In [93]:
total_distance = test_vrp_agent(q_table, num_vehicles, capacity, customers, distance_matrix)

TypeError: unsupported operand type(s) for +=: 'int' and 'tuple'